In [76]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import os.path as op
import seaborn as sns
import matplotlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.cross_decomposition import PLSRegression
%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [16]:
exp = 'sim_4mm-betas'
masks = ['lOFC_fill_sim','hipp']
smooth = 'unsmoothed'

In [17]:
def get_condition(i):
    if i < 11:
        cond = 'b_plus'
        trial = i
    elif i < 21:
        cond = 'c_plus'
        trial = i - 10
    elif i < 31:
        cond = 'c_minus'
        trial = i - 20
    elif i < 41:
        cond = 'b_minus'
        trial = i - 30
    return cond,trial

In [ ]:
betas = {'sub':[],'mask':[],'run':[],'condition':[],'trial':[],'value':[],'voxel':[],'row':[]}
for m in masks:

    for sub in subs:
        sub_path = op.abspath('./analysis/' + exp + '/' + sub + '/reg/epi/' + smooth + '/')

        mask = op.abspath('./data/' + sub + '/masks/' + m +'.nii.gz')
        mask = nib.load(mask).get_data().astype(bool)

        for run in map(str,range(1,4)):
            run_dir = sub_path + '/run_'  + run
            if os.path.exists(run_dir):
                
                for i in range(1,41):
                    f = run_dir + '/cope' + str(i) + '_xfm.nii.gz'
                    cond, trial = get_condition(i)

                    #load stat image
                    stat = nib.load(f).get_data().astype(float)
                    stat = stat[mask]
                    
                    for n,val in enumerate(stat):
                        betas['voxel'].append(n)                        
                        betas['sub'].append(sub)
                        betas['value'].append(val)
                        betas['mask'].append(m)
                        betas['run'].append(run)
                        betas['condition'].append(cond)
                        betas['trial'].append(trial)
                        betas['row'].append(i)

betas = pd.DataFrame(betas)

In [59]:
betas = betas.set_index(['sub', 'mask','run'])


In [111]:
scores_df = {'sub':[],'score':[],'y_roi':[]}
x_roi = 'hipp'
y_roi = 'lOFC_fill_sim'
train_runs = [['1','2'],['1','3'],['2','3']]

for sub in subs:
    scores = []
    for train_set in train_runs:
        X_train = []
        Y_train = []
        for run in map(str,range(1,4)):
            X = betas.loc[(sub,x_roi,run)]
            X = X.pivot(index = 'row',columns='voxel', values='value').values

            Y = betas.loc[(sub,y_roi,run)]
            Y = Y.pivot(index = 'row',columns='voxel', values='value').values

            if run in train_set:
                X_train.append(np.copy(X))
                Y_train.append(np.copy(Y))
            else:
                X_test = np.copy(X)
                Y_test = np.copy(Y)

        X_train = np.array(X_train)
        Y_train = np.array(Y_train)

        X_train = X_train.reshape(-1, X_train.shape[-1])
        Y_train = Y_train.reshape(-1, Y_train.shape[-1])

        pls20 = PLSRegression(n_components=10,max_iter = 2000)
        pls20.fit(X_train,Y_train)
        score = pls20.score(X_test,Y_test)
        scores.append(score)
    
    scores_df['sub'].append(sub)
    scores_df['y_roi'].append(y_roi)
    scores_df['score'].append(np.mean(scores))
scores_df = pd.Dataframe(scores)